In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import traceback
import re
import string
import pandas.core.algorithms as algos
from pandas import Series
import os
import datetime as dt
import matplotlib as matplot
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil import relativedelta
import sys
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
#!pip install seaborn
#!conda install -c anaconda seaborn=0.9.0

In [2]:
### Desde nuestro github público consumimos el archivo csv originas
df=pd.read_csv('https://raw.githubusercontent.com/poncho6296/Modelo_Hackathon/main/data/ubicaciones.csv')

In [3]:
df['Multiple']=np.where(df['Frecuencia']==1, 0, 1)

In [4]:
# function to plot the selected centroids           
# function to compute euclidean distance 
def distance(p1, p2): 
    return np.sum((p1 - p2)**2) 
   
# initialization algorithm 
def initialize(data, k): 
    ''' 
    initialized the centroids for K-means++ 
    inputs: 
        data - numpy array of data points having shape (200, 2) 
        k - number of clusters  
    '''
    ## initialize the centroids list and add 
    ## a randomly selected data point to the list 
    centroids = [] 
    centroids.append(data[np.random.randint( 
            data.shape[0]), :]) 
    #plot(data, np.array(centroids)) 
   
    ## compute remaining k - 1 centroids 
    for c_id in range(k - 1): 
          
        ## initialize a list to store distances of data 
        ## points from nearest centroid 
        dist = [] 
        for i in range(data.shape[0]): 
            point = data[i, :] 
            d = sys.maxsize 
              
            ## compute distance of 'point' from each of the previously 
            ## selected centroid and store the minimum distance 
            for j in range(len(centroids)): 
                temp_dist = distance(point, centroids[j]) 
                d = min(d, temp_dist) 
            dist.append(d) 
              
        ## select data point with maximum distance as our next centroid 
        dist = np.array(dist) 
        next_centroid = data[np.argmax(dist), :] 
        centroids.append(next_centroid) 
        dist = [] 
        #plot(data, np.array(centroids)) 
    return centroids 
   
# call the initialize function to get the centroids 

NameError: name 'data' is not defined

In [63]:
def estep(X:np.ndarray,centroid:np.ndarray,frecuencia=0):
    """E-step: Assigns each datapoint to the gaussian component with the
    closest mean

    Args:
        X: (n, d) array holding the data
        mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples

        """
    n,_=X.shape
    K,_= centroid.shape
    post = np.zeros((n, K))
    

    for i in range(n):
        freq=frecuencia[i]
        tiled_vector = np.tile(X[i, :], (K, 1))
        sse = ((tiled_vector - centroid)**2).sum(axis=1)
        if freq==1:
            j = np.argmin(sse)
            post[i, j] = 1
        else:
            for k in range (freq):
                j=np.argmin(sse)
                post[i,j]=1
                sse[j]=1e6

    return post

In [6]:
def mstep(X: np.ndarray, post: np.ndarray,weight=0):
    """M-step: Updates the gaussian mixture. Each cluster
    yields a component mean and variance.

    Args: X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        float: the distortion cost for the current assignment
    """
    n, d = X.shape
    _, K = post.shape
    

    n_hat = (post*Y).sum(axis=0)
    p = n_hat / n

    cost = 0
    centroid = np.zeros((K, d))
    
    for j in range(K):
        centroid[j, :] = post[:, j] @(X*Y) / n_hat[j]
        sse = ((centroid[j] - X)**2).sum(axis=1) @ post[:, j]
        cost += sse
    return centroid, cost

In [7]:
def run(X=0, centroid=0,frecuencia=0,weight=0):
    """Runs the mixture model

    Args:
        X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: distortion cost of the current assignment
    """

    prev_cost = None
    cost = None
    while (prev_cost is None or prev_cost - cost > 1e-10):
        prev_cost = cost
        post = estep(X, centroid,frecuencia)
        centroid, cost = mstep(X, post,weight)

    return centroid, post, cost

In [10]:
def normalize(v,ord=None):
    norm = np.linalg.norm(v,ord)
    if norm == 0: 
       return v
    return v / norm

### Just weighted

In [13]:
X=df[['lat','lon']].values
initial=np.array(initialize(X, k = 6) )
df['Vol_total']=df['Vol_Entrega']*df['Frecuencia']
#Ponderando con el max value
#Y=df['Vol_total'].values.reshape(X.shape[0],1)/np.max(df['Vol_total'])+\
#df['Frecuencia'].values.reshape(X.shape[0],1)/np.max(df['Frecuencia'])
#Solo metiendo como weight el peso
#Y=df['Vol_total'].values.reshape(X.shape[0],1)
#Metiendo como weight la frecuencia
#Y=df['Vol_total'].values.reshape(X.shape[0],1)
#Normalizando
Y=normalize(df['Vol_total'].values.reshape(X.shape[0],1))+\
normalize(df['Frecuencia'].values.reshape(X.shape[0],1))
f=df['Frecuencia'].values
centroid,post,cost=run(X,initial,f,Y)
zone=df
zone['D1']=post[:,0]
zone['D2']=post[:,1]
zone['D3']=post[:,2]
zone['D4']=post[:,3]
zone['D5']=post[:,4]
zone['D6']=post[:,5]
#Vamos a expxortar el csv
zone[['Id_Cliente','D1','D2','D3','D4','D5','D6']].to_csv(r'C:\Users\Jose Juarez\Documents\GitHub\Modelo_Hackathon\Output\norm.csv', index = False)
#Vamos a verlo visualmente
zone['Cluster']=np.argmax(post,axis=1)
zone=zone.melt(id_vars=['Id_Cliente', 'id_Agencia', 'Frecuencia', 'Vol_Entrega', 'lat', 'lon','Vol_total',
       'Multiple', 'Cluster'], 
        var_name="Agrupación", 
        value_name="Value")
zone=zone[zone['Value']!=0]
print(zone[['Agrupación','Vol_Entrega']].groupby(['Agrupación']).sum())
print(zone[['Agrupación','Vol_Entrega']].groupby(['Agrupación']).count())
map_Zonas = folium.Map(location=[20.506052, -98.212377], zoom_start=12)
markers_colors = []
i=0
x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, cluster in zip(zone['lat'], zone['lon'],zone['Cluster']):
    
    #i=i+1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_Zonas)
    #if i==6:
        #break
map_Zonas

             Vol_Entrega
Agrupación              
D1           7412.000000
D2           9001.833333
D3           8613.666667
D4          11133.833333
D5           9479.666667
D6           8759.000000
            Vol_Entrega
Agrupación             
D1                  617
D2                  638
D3                  597
D4                  924
D5                  579
D6                  622


In [32]:
n,_=X.shape
K,_= centroid.shape
post = np.zeros((n, K))
reorganize=np.zeros((n, K))
freq=f[0]
tiled_vector = np.tile(X[0, :], (K, 1))
sse = ((tiled_vector - centroid)**2).sum(axis=1)
reorganize[0,:]=sse
reorganize[0,:]
visits=np.zeros((1,K))
visits

array([[0., 0., 0., 0., 0., 0.]])

In [52]:
import heapq


In [ ]:
heapq.nsmallest
visits=np.zeros((1,K))


In [ ]:
def reorganize(sse,post,visit,threshold_visit,k,n,frecuencia):
    while max(visits)>=threshold_visit:
        #First select the cluster with the highest number of visits
        maxim=np.argmax(visit)
        #Know we need to organize the cluster[maxim] by the 2nd less value 
        
        for j in range(k):
            a=np.argmin(visit)
            if visit[a]<threshold_visit:
                for i in range(n):
                    if f[i]>1:
                        post[i,j]=post[i,j]
            elif:
                
    return post

In [41]:
def estep(X:np.ndarray,centroid:np.ndarray,frecuencia=0):
    """E-step: Assigns each datapoint to the gaussian component with the
    closest mean

    Args:
        X: (n, d) array holding the data
        mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples

        """
    n,_=X.shape
    K,_= centroid.shape
    post = np.zeros((n, K))
    reorganize=np.zeros((n, K))
    visits=np.zeros((1,K))
    #How many misleading points are we willing to accept threshold
    threshold_visit=1.15*int(f.sum()/K)

    for i in range(n):
        freq=frecuencia[i]
        tiled_vector = np.tile(X[i, :], (K, 1))
        sse = ((tiled_vector - centroid)**2).sum(axis=1)
        reorganize[i,:]=sse
        if freq==1:
            j = np.argmin(sse)
            post[i, j] = 1
            visits[0][j]+=1
        else:
            for k in range (freq):
                j=np.argmin(sse)
                post[i,j]=1
                sse[j]=1e6

    return post,reorganize,visits

In [43]:
post,reorganize,visits=estep(X,initial,f)

In [ ]:
def mstep(X: np.ndarray, post: np.ndarray,weight=0):
    """M-step: Updates the gaussian mixture. Each cluster
    yields a component mean and variance.

    Args: X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        float: the distortion cost for the current assignment
    """
    n, d = X.shape
    _, K = post.shape
    

    n_hat = (post*Y).sum(axis=0)
    p = n_hat / n

    cost = 0
    centroid = np.zeros((K, d))
    
    for j in range(K):
        centroid[j, :] = post[:, j] @(X*Y) / n_hat[j]
        sse = ((centroid[j] - X)**2).sum(axis=1) @ post[:, j]
        cost += sse
    return centroid, cost

In [ ]:
def run(X=0, centroid=0,frecuencia=0,weight=0):
    """Runs the mixture model

    Args:
        X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: distortion cost of the current assignment
    """

    prev_cost = None
    cost = None
    while (prev_cost is None or prev_cost - cost > 1e-10):
        prev_cost = cost
        post = estep(X, centroid,frecuencia)
        centroid, cost = mstep(X, post,weight)

    return centroid, post, cost

In [53]:
!pip install k-means-constrained
from k_means_constrained import KMeansConstrained

In [104]:
Optimizar=np.array(list(range(1,20)))/100
Optimizar=Optimizar.tolist()
for k in Optimizar:
    maxi=int((1+i)*f.sum()/6)
    mini=int((1-i)*f.sum()/6)
    clf = KMeansConstrained(n_clusters=6, size_min=mini, size_max=maxi, random_state=0).fit(X,Y)
    centers=clf.cluster_centers_
    post=estep(X,centers,f)
    zone=df
    zone['D1']=post[:,0]
    zone['D2']=post[:,1]
    zone['D3']=post[:,2]
    zone['D4']=post[:,3]
    zone['D5']=post[:,4]
    zone['D6']=post[:,5]
#Vamos a expxortar el csv
    path=r"C:\Users\Jose Juarez\Documents\GitHub\Modelo_Hackathon\Output\constrained"+str(k)+'.csv'
    zone[['Id_Cliente','D1','D2','D3','D4','D5','D6']].to_csv(path, index = False)
#Vamos a verlo visualmente
    zone['Cluster']=np.argmax(post,axis=1)
    zone=zone.melt(id_vars=['Id_Cliente', 'id_Agencia', 'Frecuencia', 'Vol_Entrega', 'lat', 'lon','Vol_total',
       'Multiple', 'Cluster'], 
        var_name="Agrupación", 
        value_name="Value")
    zone=zone[zone['Value']!=0]
    print(zone[['Agrupación','Vol_Entrega']].groupby(['Agrupación']).sum())
    print(zone[['Agrupación','Vol_Entrega']].groupby(['Agrupación']).count())

ValueError: The product of size_min and n_clusters cannot exceed the number of samples (X)

In [103]:
Optimizar=np.array(list(range(1,20)))/100
Optimizar=Optimizar.tolist()
Optimizar

[0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19]